In [1]:
import gensim
import numpy as np
import pandas as pd
import pysolr
from gensim.models import Doc2Vec

np.random.seed(42)
import tensorflow as tf
import sys

In [2]:
# In case your sys.path does not contain the base repo, go there.
print(sys.path)
%cd '/Users/axelsirota/repos/ml-solr-course'

['/Users/axelsirota/repos/ml-solr-course/3-query-generation/lab7/solutions', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python37.zip', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7/lib-dynload', '', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/IPython/extensions', '/Users/axelsirota/.ipython']
/Users/axelsirota/repos/ml-solr-course


In [3]:
model_path = '3-query-generation/lab6/alternative_queries'
query = 'Midtown sunny two bedroom'
alternative_queries_model = tf.saved_model.load(model_path)
print(f'Model loaded')


2021-07-23 22:39:41.253211: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model loaded


2021-07-23 22:39:41.890422: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


In [4]:
# Create a client instance. The timeout and authentication options are not required.
solr = pysolr.Solr('http://localhost:8983/solr/airbnb', always_commit=True, timeout=10)

In [5]:
non_expanded_results = solr.search(query, **{
                'rows': 100,
            })

In [28]:
i = 0
for result in non_expanded_results:
    i +=1
    if i == 10:
        break
    print(f'The Neighborhood is {result["neighbourhood_cleansed"]} and title is {result["name"]}')


The Neighborhood is Harlem and title is Super Large Bedroom in Harlem
The Neighborhood is Harlem and title is Sunny 2 Bedroom Penthouse with Good Vibes
The Neighborhood is East Harlem and title is Sunny Bedroom Steps to Subway & Central Park
The Neighborhood is Midtown and title is Lovely Room/ private Bath in Prewar Midtown Gem
The Neighborhood is Jackson Heights and title is Bedroom 5 min away from E, 7, F, M, R trains!
The Neighborhood is Ditmars Steinway and title is Large 2bd 15 min to Midtown Manhattan
The Neighborhood is Two Bridges and title is *Perfect Bedroom! Large Sunny Apt*Manhattan*NYC
The Neighborhood is Midtown and title is Spacious & Bright: 1 BR Midtown!
The Neighborhood is Carroll Gardens and title is Quiet Room in Unique Carroll Gardens


In [11]:
states = None
next_char = tf.constant([query])
alternative_query = []

for n in range(75):
  next_char, states = alternative_queries_model.generate_one_step(next_char, states=states)
  alternative_query.append(next_char)

print(tf.strings.join(alternative_query)[0].numpy().decode("utf-8"))


...Horatte, mutition an yull, deol puurtor ametull dewwy, the Thy kitch a B


In [12]:
new_query = ' '.join([query, ' ', tf.strings.join(alternative_query)[0].numpy().decode("utf-8")])

In [13]:
new_query

'Midtown sunny two bedroom   ...Horatte, mutition an yull, deol puurtor ametull dewwy, the Thy kitch a B'

In [14]:
expanded_results = solr.search(new_query, **{
                'rows': 100,
            })

In [29]:
i = 0
for result in expanded_results:
    i +=1
    if i == 10:
        break
    print(f'The Neighborhood is {result["neighbourhood_cleansed"]} and title is {result["name"]}')

The Neighborhood is Brighton Beach and title is The best place to rest and relax
The Neighborhood is Park Slope and title is Massive, Private 1 Bedroom near Barclays Center
The Neighborhood is Harlem and title is Super Large Bedroom in Harlem
The Neighborhood is Hell's Kitchen and title is Spacious 2 bedroom near Times Sq
The Neighborhood is Midtown and title is Spacious & Bright: 1 BR Midtown!
The Neighborhood is Harlem and title is Sunny 2 Bedroom Penthouse with Good Vibes
The Neighborhood is Bushwick and title is Beautiful Bushwick - 3BR Apartment
The Neighborhood is Maspeth and title is 2 bdrm  Entire Furnished Apt.  (R)
The Neighborhood is Midtown and title is Large Elevator 1 Bedroom Best Location


In [16]:
model_path = '2-ranking/lab4/airbnb_model'
doc2vec_model = Doc2Vec.load(model_path)
print(f'Doc2Vec Model loaded')

Doc2Vec Model loaded


In [17]:
tokenized_query = list(gensim.utils.simple_preprocess(query))
tokenized_new_query = list(gensim.utils.simple_preprocess(new_query))

In [18]:
tokenized_new_query

['midtown',
 'sunny',
 'two',
 'bedroom',
 'horatte',
 'mutition',
 'an',
 'yull',
 'deol',
 'puurtor',
 'ametull',
 'dewwy',
 'the',
 'thy',
 'kitch']

In [19]:
df_non_expanded_results = pd.DataFrame(non_expanded_results)
similarities = []
for result in non_expanded_results:
    try:
        similarity = doc2vec_model.similarity_unseen_docs(doc_words1=tokenized_query, doc_words2= list(gensim.utils.simple_preprocess(result["description"])))
    except KeyError:
        similarity = 0
    similarities.append(similarity)
df_non_expanded_results["Similarity"] = pd.Series(similarities)
df_non_expanded_results.sort_values(by="Similarity", ascending=False, inplace=True)


In [20]:
df_non_expanded_results

,id,listing_url,name,description,neighborhood_overview,host_id,host_url,host_name,neighbourhood,neighbourhood_cleansed,...,accommodates,bathrooms_text,bedrooms,beds,amenities,price,number_of_reviews,reviews_per_month,_version_,Similarity
44,48102360,https://www.airbnb.com/rooms/48102360,Convenient to all transportation,Beautiful sunny quiet upper West side two bedr...,NaN,13481312,https://www.airbnb.com/users/show/13481312,Susan,NaN,Upper West Side,...,4,1 bath,2.0,2.0,"[""Kitchen"", ""Cable TV"", ""TV"", ""Carbon monoxide...",$150.00,0,NaN,1706113115790245890,0.838188
73,45989074,https://www.airbnb.com/rooms/45989074,Two Bedroom Apartment in Midtown west,Bright Two bedroom apartment located in Midtow...,NaN,51501835,https://www.airbnb.com/users/show/51501835,Jeniffer,NaN,Hell's Kitchen,...,4,1 bath,2.0,1.0,"[""Kitchen"", ""TV"", ""Carbon monoxide alarm"", ""Es...",$90.00,1,[0.6],1706113115179974658,0.804656
14,19160912,https://www.airbnb.com/rooms/19160912,Spacious Guest room 3 blocks from NY Presbyter...,"Spacious guest bedroom available in sunny, cha...",NaN,1728792,https://www.airbnb.com/users/show/1728792,Brittany,NaN,Washington Heights,...,2,1 shared bath,1.0,1.0,"[""Hot water"", ""Essentials"", ""Smoke alarm"", ""Wi...",$76.00,4,[0.16],1706113110340796417,0.775606
39,45732541,https://www.airbnb.com/rooms/45732541,Central Williamsburg location ! Close to L and J,Sunny two bedroom apartment close to coffee sh...,NaN,101573638,https://www.airbnb.com/users/show/101573638,Natasha,NaN,Williamsburg,...,2,1 shared bath,1.0,1.0,"[""Kitchen"", ""Hot water"", ""Essentials"", ""Heatin...",$65.00,11,[1.82],1706113115075117058,0.757887
79,46432262,https://www.airbnb.com/rooms/46432262,Two Bedroom Apartment in Midtown West,Two bedroom apartment in Midtown West,NaN,51501835,https://www.airbnb.com/users/show/51501835,Jeniffer,NaN,Hell's Kitchen,...,4,1 bath,2.0,1.0,"[""Kitchen"", ""TV"", ""Carbon monoxide alarm"", ""Ha...",$150.00,0,NaN,1706113115320483842,0.756615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,27903031,https://www.airbnb.com/rooms/27903031,Lovely Room/ private Bath in Prewar Midtown Gem,Very well located elegant 3 bedroom sunny prew...,This is a great family apartment in the middle...,7580102,https://www.airbnb.com/users/show/7580102,Jamie,"New York, United States",Midtown,...,2,1 shared bath,1.0,1.0,"[""Cable TV"", ""Essentials"", ""Dryer"", ""Long term...",$175.00,5,[0.16],1706113111743791106,0.259540
78,938022,https://www.airbnb.com/rooms/938022,Private Bedroom in Williamsburg,Big sunny room with queen bed. Well located in...,Conveniently located on the eastern edge of Wi...,1866827,https://www.airbnb.com/users/show/1866827,Gregory,"Brooklyn, New York, United States",Williamsburg,...,2,NaN,1.0,1.0,"[""Refrigerator"", ""Essentials"", ""Long term stay...",$85.00,103,[1.04],1706113107109085185,0.254714
25,47423109,https://www.airbnb.com/rooms/47423109,"Bohemian style, New Unit in Prime location",Bohemian and modern and perfect location! Quie...,Midtown East - Prime location,51589519,https://www.airbnb.com/users/show/51589519,Asi,"New York, United States",Midtown,...,2,1 bath,NaN,1.0,"[""Cable TV"", ""Refrigerator"", ""Shower gel"", ""Es...",$125.00,1,[1.0],1706113115622473728,0.230108
19,40897147,https://www.airbnb.com/rooms/40897147,Sunny Astoria,Sunny private room in a two-bedroom apartment ...,Astoria is one of the hottest neighborhoods in...,25768880,https://www.airbnb.com/users/show/25768880,Andi,"Queens, New York, United States",Astoria,...,1,1 shared bath,1.0,1.0,"[""Hair dryer"", ""Shampoo"", ""Kitchen"", ""Fire ext...",$45.00,1,[0.06],1706113113927974916,0.200648


In [21]:
print(f'Most similar document before expansion has description: \n\n{df_non_expanded_results["description"].iloc[0]}\nWith similarity: {df_non_expanded_results["Similarity"].iloc[0]}')

Most similar document before expansion has description: 

Beautiful sunny quiet upper West side two bedroom apartment.
With similarity: 0.8381876945495605


In [22]:
df_expanded_results = pd.DataFrame(expanded_results)
new_similarities = []
for result in expanded_results:
    try:
        similarity = doc2vec_model.similarity_unseen_docs(doc_words1=tokenized_query, doc_words2= list(gensim.utils.simple_preprocess(result["description"])))
    except KeyError:
        similarity = 0
    new_similarities.append(similarity)
df_expanded_results["Similarity"] = pd.Series(new_similarities)
df_expanded_results.sort_values(by="Similarity", ascending=False, inplace=True)

In [23]:
print(f'Most similar document after expansion has description: \n\n{df_expanded_results["description"].iloc[0]}\nWith similarity: {df_expanded_results["Similarity"].iloc[0]}')

Most similar document after expansion has description: 

A sunny, spacious and furnished two bedroom apartment with skylight and balcony. A quick walk to the JMZ subway lines with an express train to Manhattan.
With similarity: 0.7855047583580017


In [24]:
print(f'Number of documents that surpass 0.5 similarity threshold: {len(df_expanded_results[df_expanded_results["Similarity"] >= 0.5])}')


Number of documents that surpass 0.5 similarity threshold: 32
